Testing the complexity of u_sin implemented by `amplitude encoding assignment`

In [1]:
import classiq
from classiq import *
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq.qmod.symbolic import sin, cos
print("Classiq version:", classiq.__version__)

Classiq version: 0.72.0


In [4]:
def get_metrics(qprog):
    """
    Extract circuit metrics from a quantum program.

    Parameters:
        qprog: The quantum program object.

    Returns:
        dict: A dictionary containing the circuit metrics:
              - "depth": Circuit depth.
              - "width": Circuit width (number of qubits used).
              - "cx_count": Number of CX gates (returns 0 if none are present).
    """
    # Generate the optimized quantum circuit
    circuit = QuantumProgram.from_qprog(qprog)

    # Extract metrics
    metrics = {
        "depth": circuit.transpiled_circuit.depth,
        "width": circuit.data.width,
        "cx_count": circuit.transpiled_circuit.count_ops.get('cx', 0)  # Default to 0 if 'cx' not found
    }

    return metrics

def create_classis_program(num_qubits):
    @qfunc
    def u_sin(x: QNum, a: QNum) -> None:
        a *= sin(x / (2**num_qubits))  # Amplitude encoding sin(x) to |1>
        X(a)  # sin(x) to |0>


    @qfunc
    def main(x: Output[QNum], ind: Output[QNum]):
        allocate(num_qubits, x)
        allocate(1, ind)
        u_sin(x, ind)

    qmod = create_model(
        main,
        constraints=Constraints(max_width=100),
        execution_preferences=ExecutionPreferences(
            num_shots=1,
            backend_preferences=ClassiqBackendPreferences(
                backend_name="simulator_statevector"
            ),
        ),
        # out_file="qsvt",
    )
    qprog = synthesize(qmod)
    return qprog

In [5]:
for nqbits in range(2, 10):
    qprog = create_classis_program(num_qubits=nqbits)
    metrics = get_metrics(qprog)
    print(f"Metrics for {nqbits} qubits: {metrics}")

Metrics for 2 qubits: {'depth': 7, 'width': 3, 'cx_count': 4}
Metrics for 3 qubits: {'depth': 12, 'width': 4, 'cx_count': 8}
Metrics for 4 qubits: {'depth': 21, 'width': 5, 'cx_count': 16}
Metrics for 5 qubits: {'depth': 38, 'width': 6, 'cx_count': 32}
Metrics for 6 qubits: {'depth': 71, 'width': 7, 'cx_count': 64}
Metrics for 7 qubits: {'depth': 136, 'width': 8, 'cx_count': 128}
Metrics for 8 qubits: {'depth': 265, 'width': 9, 'cx_count': 256}
Metrics for 9 qubits: {'depth': 522, 'width': 10, 'cx_count': 512}
